In [1]:
import os,sys
import math
import numpy as np

import PIL
import torchvision
import torchvision.transforms as T

import torch
import torch.nn as nn

In [2]:
print(torch.__version__)

1.9.0+cu102


## reference: [Logistic Regression with PyTorch](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_logistic_regression/) and [Basics of Neural Network](http://www.ai-start.com/dl2017/html/lesson1-week2.html)

## 1. Logistic regression and softmax

<img src="./SchematicImages/logistic_regression_1.png" alt="drawing" width="600"/>

## 2. A logistic regression case

### 2.1 Loading training and test dataset and make it iterable

In [3]:
class Dataset(torch.utils.data.Dataset):
    'Characterizes an image dataset for PyTorch'
    def __init__(self, list_files, labels, transform = None):
        'Initialization'
        self.labels = labels
        self.list_files = list_files
        self.transform = transform
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_files)
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        file_dir = self.list_files[index]

        # Load data and get label
        image = PIL.Image.open(file_dir)
        if self.transform:
            image = self.transform(image)
        X = image
        y = torch.tensor(self.labels[file_dir])

        return X, y

data_dir = "/mnt/Storage/home/yuzhaowei/.keras/datasets/flower_photos"
labels = {}
daisy_images = [os.path.join((data_dir), "daisy") + "/" + i for i in os.listdir(os.path.join((data_dir), "daisy"))]
rose_images = [os.path.join((data_dir), "roses") + "/" + i for i in os.listdir(os.path.join((data_dir), "roses"))]
sunflowers_images = [os.path.join((data_dir), "sunflowers") + "/" + i for i in os.listdir(os.path.join((data_dir), "sunflowers"))]
samples = np.concatenate([np.random.choice(daisy_images, 500), np.random.choice(rose_images, 500), np.random.choice(sunflowers_images, 500)])

for image in daisy_images:
    labels[image] = 0 # dasiy
for image in rose_images:
    labels[image] = 1 # roses
for image in sunflowers_images:
    labels[image] = 2 # sunflowers

class Rescale():
    """Reize image in a sample to a given size"""
    def __call__(self, image):
        image_resize = np.array(T.Resize(size = (224, 224))(image))
        return(image_resize)
    
# Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] if the PIL Image belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) or if the numpy.ndarray has dtype = np.uint8
composed = T.Compose([Rescale(), T.ToTensor()]) # composed two treatments for input images    
        
    
dataset = Dataset(samples, labels, transform = composed)
train_set, test_set = torch.utils.data.random_split(dataset, lengths = [int(1500 * 0.8), int(1500 * 0.2)])

In [4]:
train_set[0] 

(tensor([[[0.9961, 0.9882, 0.9843,  ..., 0.7529, 0.8039, 0.8824],
          [0.9961, 0.9882, 0.9843,  ..., 0.8667, 0.8980, 0.9098],
          [0.9961, 0.9882, 0.9843,  ..., 0.9176, 0.9137, 0.8980],
          ...,
          [0.5843, 0.5647, 0.5451,  ..., 0.9216, 0.9255, 0.9255],
          [0.6000, 0.5804, 0.5647,  ..., 0.9294, 0.9216, 0.9216],
          [0.5922, 0.5843, 0.5569,  ..., 0.9294, 0.9255, 0.9294]],
 
         [[0.7843, 0.7961, 0.8000,  ..., 0.6941, 0.7216, 0.8196],
          [0.7843, 0.7961, 0.8000,  ..., 0.8078, 0.8392, 0.8667],
          [0.7882, 0.8000, 0.8000,  ..., 0.8549, 0.8549, 0.8510],
          ...,
          [0.6667, 0.6510, 0.6275,  ..., 0.8588, 0.8627, 0.8627],
          [0.6667, 0.6549, 0.6314,  ..., 0.8667, 0.8667, 0.8667],
          [0.6706, 0.6510, 0.6196,  ..., 0.8627, 0.8667, 0.8627]],
 
         [[0.6863, 0.7020, 0.7216,  ..., 0.6549, 0.7020, 0.8078],
          [0.6863, 0.7020, 0.7216,  ..., 0.7608, 0.8039, 0.8314],
          [0.6902, 0.7059, 0.7216,  ...,

In [5]:
# dataset size
print(len(train_set))
print(len(test_set))

1200
300


When the model goes through the whole 1200 images once, learning how to classify 3 groups of flowers, it's consider 1 epoch. 

However, there's a concept of batch size where it means the model would look at 100 images before updating the model's weights, thereby learning. When the model updates its weights (parameters) after looking at all the images, this is considered 1 iteration.

In [6]:
batch_size = 120

We arbitrarily set 100 iterations here which means the model would update 100 times.

In [7]:
n_iters = 100

Because we would like to go through 100 iterations, this implies we would have 100 / 1200 * 120 = 10 epochs as each epoch has 10 iterations.

In [8]:
num_epochs = n_iters / (len(train_set) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

10


In [9]:
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 6,
         'drop_last' : True # set to True to drop the last incomplete batch, if the dataset size is not divisible by the batch size.
         }
train_generator = torch.utils.data.DataLoader(train_set, **params)
test_generator = torch.utils.data.DataLoader(test_set, **params)

import collections
isinstance(train_generator, collections.Iterable)
isinstance(test_generator, collections.Iterable)

<ipython-input-9-e787f1fbcdca>:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  isinstance(train_generator, collections.Iterable)


True

### 2.2 building model

In [10]:
# Same as linear regression! 
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        # And we're feeding the model with n*n images, and output are number of classification groups.
        self.linear = nn.Linear(input_dim, output_dim)  # input dim = 3 * 224 * 224, output_dim = 3

    def forward(self, x):
        out = self.linear(x)
        return(out)
    
input_dim = 3 * 224 * 224
output_dim = 3

model = LogisticRegressionModel(input_dim, output_dim)

### 2.3 instantiate loss class (logistic regression: cross entropy loss)

nn.CrossEntropyLoss() does 2 things at the same time.

1. Computes softmax (logistic/softmax function)
2. Computes cross entropy

In [11]:
criterion = nn.CrossEntropyLoss()

### 2.4 instantiate optimizer class

calculates the parameters' gradients and update them subsequently.

In [12]:
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

### 2.5 train model

Process:
    1. Convert inputs/labels to tensors with gradients
    2. Clear gradient buffets
    3. Get output given inputs
    4. Get loss
    5. Get gradients w.r.t. parameters
    6. Update parameters using gradientsparameters = parameters - learning_rate * parameters_gradients
    7. REPEAT

In [15]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_generator):
        # Load images as Variable
        images = images.view(-1, 3*224*224).requires_grad_()
        # images = images.requires_grad_()
        labels = labels
        # print(labels)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)
        # print(outputs)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 10 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_generator:
                # Load images to a Torch Variable
                images = images.view(-1, 3*224*224).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 10. Loss: 3.3807289600372314. Accuracy: 51.25
Iteration: 20. Loss: 2.110201120376587. Accuracy: 55.83333206176758
Iteration: 30. Loss: 0.7504737973213196. Accuracy: 66.25
Iteration: 40. Loss: 0.6078311204910278. Accuracy: 67.08333587646484
Iteration: 50. Loss: 2.7222421169281006. Accuracy: 58.75
Iteration: 60. Loss: 0.4933057427406311. Accuracy: 68.33333587646484
Iteration: 70. Loss: 1.7783819437026978. Accuracy: 57.91666793823242
Iteration: 80. Loss: 0.9139341711997986. Accuracy: 64.58333587646484
Iteration: 90. Loss: 3.2309868335723877. Accuracy: 46.25
Iteration: 100. Loss: 1.1253330707550049. Accuracy: 64.58333587646484


In [14]:
images

tensor([[0.7373, 0.7490, 0.7647,  ..., 0.4353, 0.4392, 0.4431],
        [0.8039, 0.7961, 0.8039,  ..., 0.0196, 0.0235, 0.0235],
        [0.6863, 0.6863, 0.6863,  ..., 0.2549, 0.2549, 0.2549],
        ...,
        [0.5686, 0.5765, 0.5843,  ..., 0.2118, 0.2078, 0.2000],
        [0.6392, 0.6431, 0.6431,  ..., 0.6784, 0.6784, 0.6784],
        [1.0000, 0.9922, 0.9961,  ..., 1.0000, 1.0000, 1.0000]],
       requires_grad=True)

In [34]:
train_set[0]

(tensor([[[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1]],
 
         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1]],
 
         [[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1]]], dtype=torch.uint8),
 'roses')

In [9]:
train_set[0]

(tensor([[[193, 193, 193,  ..., 108, 108, 106],
          [190, 190, 190,  ..., 108, 108, 106],
          [187, 186, 186,  ..., 108, 108, 106],
          ...,
          [ 91,  89,  88,  ...,  76,  73,  73],
          [ 90,  88,  88,  ...,  76,  73,  73],
          [ 90,  88,  88,  ...,  76,  73,  73]],
 
         [[183, 183, 183,  ...,  84,  84,  82],
          [180, 180, 180,  ...,  84,  84,  82],
          [177, 176, 176,  ...,  84,  84,  82],
          ...,
          [ 90,  88,  87,  ...,  58,  57,  57],
          [ 89,  87,  87,  ...,  58,  57,  57],
          [ 89,  87,  87,  ...,  58,  57,  57]],
 
         [[173, 173, 173,  ...,  58,  58,  56],
          [170, 170, 170,  ...,  58,  58,  56],
          [167, 166, 166,  ...,  58,  58,  56],
          ...,
          [ 60,  58,  57,  ...,  21,  21,  21],
          [ 59,  57,  57,  ...,  21,  21,  21],
          [ 59,  57,  57,  ...,  21,  21,  21]]], dtype=torch.uint8),
 'daisy')

array([[[ 1, 35,  0],
        [25, 39,  3],
        [27, 44,  0],
        ...,
        [29, 45,  0],
        [25, 41,  2],
        [21, 38,  2]],

       [[11, 32,  0],
        [31, 39,  0],
        [41, 50,  3],
        ...,
        [35, 54,  0],
        [25, 47,  1],
        [18, 39,  0]],

       [[13, 38,  0],
        [29, 48,  2],
        [45, 57,  7],
        ...,
        [40, 62,  0],
        [27, 51,  1],
        [18, 41,  0]],

       ...,

       [[16, 38,  2],
        [15, 42,  1],
        [13, 46,  3],
        ...,
        [ 1, 27,  0],
        [ 0, 21,  0],
        [ 0, 15,  0]],

       [[12, 33,  2],
        [10, 36,  0],
        [10, 41,  0],
        ...,
        [ 2, 28,  1],
        [ 1, 20,  0],
        [ 1, 15,  0]],

       [[16, 36,  0],
        [12, 33,  0],
        [10, 34,  2],
        ...,
        [ 1, 27,  0],
        [ 0, 24,  0],
        [ 0, 20,  0]]], dtype=uint8)